In [14]:
Threads.nthreads()

16

## Distributed package # works

In [22]:
using Distributed
n = 1e8
@time @sync nheads1 = @distributed (+) for i = 1:n; rand(Bool); end

  1.954359 seconds (527.84 k allocations: 26.720 MiB)


50004879

In [10]:
nheads2 = 0;
@time for i = 1:n; global nheads2+=rand(Bool); end
println(nheads2)

 22.553226 seconds (400.00 M allocations: 7.451 GiB, 3.59% gc time)
49993103


### Distribute workers within for loop (Experimental only!)

In [32]:
n = 1e7

@everywhere s = 0
@time Threads.@threads for i = 1:n
    global s
    s+=rand(Bool)
end
println(s)

  3.685583 seconds (2.60 M allocations: 43.649 MiB)
516755


In [33]:
sum = 0
@time for i = 1:n
    global sum
    sum+=rand(Bool)
end
println(sum)

  3.470345 seconds (40.00 M allocations: 762.916 MiB)
4999000


In [35]:
## Show what threads that is involved
n = 32
a = zeros(n);
Threads.@threads for i = 1:n
    a[i] = Threads.threadid()
end
println(a)

[1.0, 1.0, 2.0, 2.0, 3.0, 3.0, 4.0, 4.0, 5.0, 5.0, 6.0, 6.0, 7.0, 7.0, 8.0, 8.0, 9.0, 9.0, 10.0, 10.0, 11.0, 11.0, 12.0, 12.0, 13.0, 13.0, 14.0, 14.0, 15.0, 15.0, 16.0, 16.0]


## Map out distributed jobs

In [1]:
using Distributed
addprocs(1)
rmprocs(workers())
n_proc = 16
addprocs(n_proc); # add number of worker processes
@everywhere basepath = "/home/dreuter/Github/julia-paths"
@everywhere thispath = "/LearningPath-Julia/"
@everywhere fullpath = string(basepath,thispath)

In [2]:
@everywhere include(string(fullpath,"Counter.jl"))

In [8]:
Counter.count_heads(200)

94

In [9]:
function parallel_map(n,n_par)
    joblist = []
    n_job = Int(round(n/n_par))
    for i = 1:n_par
        push!(joblist, @spawn(Counter.count_heads(n_job)))
    end
    return joblist
end;

In [39]:
n = 2e10;
n_par = 16;

## Compare with and without parallell jobs

In [40]:
joblist = parallel_map(n,n_par);
@time s = sum(map(fetch, joblist));
println(round(s, digits=2))

  4.903298 seconds (1.53 k allocations: 53.281 KiB)
1.0000050271e10


In [38]:
@time s = Counter.count_heads(n);
println(round(s, digits=2))

  5.143058 seconds (5 allocations: 176 bytes)
1.000009205e9


## RemoteChannels

In [1]:
using Distributed
addprocs(1)
rmprocs(workers())
addprocs(16); # add number of worker processes

@everywhere function count_heads(n)
            c::Int = 0
            for i = 1:n
                c += rand(Bool)
            end
            return c
        end

@everywhere function do_flip_work(jobs, results, flips) # define work function everywhere
            while true
                job_id = take!(jobs)
                nheads = count_heads(flips)
                put!(results, (job_id, nheads, myid()))
            end
        end

function make_jobs(n)
    for i in 1:n
        put!(jobs, i)
    end
end;

function execute_tasks(n, flips)
    @async make_jobs(n); # feed the jobs channel with "n" jobs
    for p in workers() # start tasks on the workers to process requests in parallel
        remote_do(do_flip_work, p, jobs, results, flips)
    end
end

execute_tasks (generic function with 1 method)

In [7]:
jobs = RemoteChannel(()->Channel{Int}(32));
results = RemoteChannel(()->Channel{Tuple}(32));

n_jobs = 16; # add number of jobs
n_flips = 2e10
execute_tasks(n_jobs, n_flips/n_jobs)

# Display overall process and results
s = 0;
@time while n_jobs > 0 # print out results
    global s
    job_id, nheads, where = take!(results)
    println("Job $job_id counts to $(round(nheads; digits=2)) heads on worker $where")
    s += nheads
    global n_jobs -= 1
end;
println("The total number of heads is: $(round(s; digits=2))");

Job 7 counts to 6.25006235e8 heads on worker 9
Job 12 counts to 6.25014227e8 heads on worker 14
Job 10 counts to 6.25002212e8 heads on worker 12
Job 6 counts to 6.24972556e8 heads on worker 8
Job 4 counts to 6.25016437e8 heads on worker 6
Job 1 counts to 6.25010946e8 heads on worker 3
Job 11 counts to 6.24965936e8 heads on worker 13
Job 9 counts to 6.24965939e8 heads on worker 11
Job 5 counts to 6.24988248e8 heads on worker 7
Job 8 counts to 6.25005487e8 heads on worker 10
Job 16 counts to 6.24987603e8 heads on worker 18
Job 2 counts to 6.25011382e8 heads on worker 4
Job 14 counts to 6.25009435e8 heads on worker 16
Job 3 counts to 6.2500268e8 heads on worker 5
Job 13 counts to 6.25008762e8 heads on worker 15
Job 15 counts to 6.24966425e8 heads on worker 17
  4.891964 seconds (3.90 k allocations: 167.969 KiB)
The total number of heads is: 9.99993451e9
